# Exploring Chord Diagrams for Arms Trade Visualization

This notebook explores how to create and customize chord diagrams using D3.js for visualizing arms trade data. We'll cover:
1. Basic chord diagram structure
2. Data preparation
3. Creating the visualization
4. Adding interactivity
5. Styling and customization

## 1. Required Libraries

First, let's import the necessary libraries for data manipulation and analysis.

In [1]:
import pandas as pd
import numpy as np
import json

# For data visualization in the notebook
import matplotlib.pyplot as plt
import seaborn as sns

## 2. Sample Data Structure

Let's create a sample dataset that represents arms trade between countries. We'll use the SIPRI TIV (Trend Indicator Value) as our measure.

In [2]:
# Load actual arms trade data from JSON file
with open('../data/arms_trade_matrices/arms_trade_matrix_2021.json', 'r') as f:
    matrix_data = json.load(f)

# Convert matrix data to sample_data format
sample_data = {
    "year": 2021,
    "trades": []
}

# Convert matrix to trades list
countries = matrix_data['countries']
matrix = matrix_data['matrix']
for i, supplier in enumerate(countries):
    for j, recipient in enumerate(countries):
        if matrix[i][j] > 0:
            sample_data['trades'].append({
                "supplier": supplier,
                "recipient": recipient, 
                "value": matrix[i][j]
            })

# Convert to DataFrame for easier manipulation
df = pd.DataFrame(sample_data['trades'])
print("Sample DataFrame:")
print(df)

Sample DataFrame:
   supplier recipient     value
0       USA   GERMANY    165.80
1       USA     OTHER  15012.78
2    RUSSIA     CHINA    130.50
3    RUSSIA     OTHER   1545.33
4        UK       USA     35.80
5        UK     OTHER    509.12
6   GERMANY     OTHER    522.20
7     CHINA     OTHER   2544.64
8     OTHER       USA    589.04
9     OTHER   GERMANY     62.75
10    OTHER     OTHER   9300.13


## 3. Data Preparation for Chord Diagram

To create a chord diagram, we need to transform our data into a matrix format where:
- Rows represent suppliers
- Columns represent recipients
- Values represent trade volumes

In [3]:
def prepare_chord_matrix(df):
    # Get unique countries (both suppliers and recipients)
    countries = sorted(list(set(df['supplier'].unique()) | set(df['recipient'].unique())))
    
    # Create empty matrix
    n = len(countries)
    matrix = np.zeros((n, n))
    
    # Fill matrix with trade values
    for _, row in df.iterrows():
        i = countries.index(row['supplier'])
        j = countries.index(row['recipient'])
        matrix[i][j] = row['value']
    
    return countries, matrix

countries, matrix = prepare_chord_matrix(df)
print("Countries:", countries)
print("\nTrade Matrix:")
print(matrix)

Countries: ['CHINA', 'GERMANY', 'OTHER', 'RUSSIA', 'UK', 'USA']

Trade Matrix:
[[    0.       0.    2544.64     0.       0.       0.  ]
 [    0.       0.     522.2      0.       0.       0.  ]
 [    0.      62.75  9300.13     0.       0.     589.04]
 [  130.5      0.    1545.33     0.       0.       0.  ]
 [    0.       0.     509.12     0.       0.      35.8 ]
 [    0.     165.8  15012.78     0.       0.       0.  ]]


## 4. Preparing D3.js Compatible Format

Now we'll create the JSON structure that our D3.js chord diagram will use.

In [4]:
def create_d3_json(countries, matrix):
    return {
        "countries": countries,
        "matrix": matrix.tolist()
    }

d3_data = create_d3_json(countries, matrix)
print(json.dumps(d3_data, indent=2))

{
  "countries": [
    "CHINA",
    "GERMANY",
    "OTHER",
    "RUSSIA",
    "UK",
    "USA"
  ],
  "matrix": [
    [
      0.0,
      0.0,
      2544.64,
      0.0,
      0.0,
      0.0
    ],
    [
      0.0,
      0.0,
      522.2,
      0.0,
      0.0,
      0.0
    ],
    [
      0.0,
      62.75,
      9300.130000000001,
      0.0,
      0.0,
      589.04
    ],
    [
      130.5,
      0.0,
      1545.33,
      0.0,
      0.0,
      0.0
    ],
    [
      0.0,
      0.0,
      509.12,
      0.0,
      0.0,
      35.8
    ],
    [
      0.0,
      165.8,
      15012.78,
      0.0,
      0.0,
      0.0
    ]
  ]
}


## 5. D3.js Chord Diagram Implementation

Below is the D3.js code that would create the chord diagram. This code would go in your React component.

In [5]:
# This is JavaScript code for reference
d3_code = """
// Create chord layout
const chord = d3.chord()
    .padAngle(0.05)
    .sortSubgroups(d3.descending);

// Create arc generator
const arc = d3.arc()
    .innerRadius(radius)
    .outerRadius(radius + 20);

// Create ribbon generator
const ribbon = d3.ribbon()
    .radius(radius);

// Create color scale
const color = d3.scaleOrdinal(d3.schemeCategory10);
"""

print("D3.js implementation reference:")
print(d3_code)

D3.js implementation reference:

// Create chord layout
const chord = d3.chord()
    .padAngle(0.05)
    .sortSubgroups(d3.descending);

// Create arc generator
const arc = d3.arc()
    .innerRadius(radius)
    .outerRadius(radius + 20);

// Create ribbon generator
const ribbon = d3.ribbon()
    .radius(radius);

// Create color scale
const color = d3.scaleOrdinal(d3.schemeCategory10);



## 6. Data Analysis Functions

Let's create some helper functions to analyze the arms trade data.

In [ ]:
def analyze_trade_flows(df):
    # Total trade volume by supplier
    supplier_totals = df.groupby('supplier')['value'].sum().sort_values(ascending=False)
    
    # Total trade volume by recipient
    recipient_totals = df.groupby('recipient')['value'].sum().sort_values(ascending=False)
    
    print("Top Suppliers:")
    print(supplier_totals)
    print("\nTop Recipients:")
    print(recipient_totals)
    
    return supplier_totals, recipient_totals

supplier_totals, recipient_totals = analyze_trade_flows(df)

## 7. Visualization Customization

Here are some examples of how to customize the chord diagram appearance.

In [ ]:
# Example color schemes and styling options
visualization_options = {
    "color_schemes": [
        "d3.schemeCategory10",
        "d3.schemeSet3",
        "d3.schemePaired"
    ],
    "styling": {
        "padAngle": 0.05,
        "innerRadius": 200,
        "outerRadius": 220,
        "labelOffset": 10,
        "ribbonOpacity": 0.6
    }
}

print(json.dumps(visualization_options, indent=2))

## 8. Interactive Features

Description of interactive features we can add to the chord diagram.

In [ ]:
interactive_features = {
    "hover_effects": [
        "Highlight related ribbons",
        "Show tooltip with trade details",
        "Fade unrelated connections"
    ],
    "click_actions": [
        "Isolate country's connections",
        "Show detailed trade information",
        "Toggle ribbon visibility"
    ],
    "animations": [
        "Smooth transitions on data updates",
        "Ribbon hover animations",
        "Year transition effects"
    ]
}

print(json.dumps(interactive_features, indent=2))

## 9. Best Practices and Recommendations

1. Data Preparation:
   - Always validate and clean the data
   - Handle missing values appropriately
   - Normalize trade values if necessary

2. Visualization:
   - Use appropriate color schemes
   - Ensure readable labels
   - Add clear legends and tooltips

3. Performance:
   - Optimize data structure
   - Use appropriate D3.js methods
   - Handle large datasets efficiently

## 10. Testing Data Transformation

Let's create a function to test our data transformation pipeline.

In [ ]:
def test_data_pipeline(sample_data):
    # Convert raw data to DataFrame
    df = pd.DataFrame(sample_data['trades'])
    
    # Create matrix
    countries, matrix = prepare_chord_matrix(df)
    
    # Create D3 format
    d3_data = create_d3_json(countries, matrix)
    
    # Validate results
    print("Validation Results:")
    print(f"Number of countries: {len(countries)}")
    print(f"Matrix shape: {matrix.shape}")
    print(f"Total trade volume: {matrix.sum()}")
    
    return d3_data

test_results = test_data_pipeline(sample_data)